In [ ]:
# 베스트 모델 찾기 - 와인 이진분류 

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
wine = pd.read_csv('dataset/wine.csv',header =None)
wine.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1


In [3]:
# LabelENcoder
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(wine.iloc[:,-1].values)

In [4]:
# One-hot encoding
from tensorflow.keras.utils import to_categorical
y_onehot = to_categorical(y)
y_onehot[:3]                                                                                                                        

array([[0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [5]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(wine.iloc[:,:-1].values)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, y_onehot, stratify=y_onehot, random_state=seed
)
X_train.shape,X_test.shape, Y_train.shape, Y_test.shape 

((4872, 12), (1625, 12), (4872, 2), (1625, 2))

### 모델 정의/설정/학습/평가

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [9]:
model = Sequential()
model.add(Dense(16,input_dim=4, activation='relu')) # Hidden layer
model.add(Dense(3, activation ='relu'))
model.add(Dense(3, activation ='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 12        
Total params: 143
Trainable params: 143
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(
    optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy']
)

### 모델 저장관련 설정 

In [13]:
modelpath = ' model/best_wine{epoch:03d}_{val_loss:.4f}.h5'

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint
Checkpoint = ModelCheckpoint(
    modelpath, monitor='val_loss', verbose = 1, save_best_only=True
)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=20)

### 모델 학습 및 저장

In [16]:
history = model.fit(
    X_train, Y_train, validation_split = 0.2,
    epochs=200, batch_size = 200, verbose=0,
    callbacks=[Checkpoint]
)

ValueError: in user code:

    C:\Users\CPB06GameN\miniconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\CPB06GameN\miniconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\CPB06GameN\miniconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\CPB06GameN\miniconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\CPB06GameN\miniconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\CPB06GameN\miniconda3\lib\site-packages\tensorflow\python\keras\e

### 평가는 베스트 모델로 해야 함

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model

In [ ]:
best_model.evaluate(X_test,y_test)